In [1]:
import numpy as np
import json
import pandas as pd
import time
import math
import matplotlib.pyplot as plt

from src.learner import *

samples = np.genfromtxt('data/samples.csv', delimiter=",").astype(bool)
holdouts = np.genfromtxt('data/holdouts.csv', delimiter=",").astype(bool)
tests = np.genfromtxt('data/tests.csv', delimiter=",").astype(bool)

In [2]:
def scale(x, K):
    return K*math.log(x)

The final hyperparameters we settled on were:

Hidden units: 100  
Learning rate: 0.01  
Batch size: 16  
Epochs: 50  

These produced a training and testing accuracy of 0.997 and 0.986, respectively, and were associated with a runtime of under one second. While slightly higher train and test accuracies could have been achieved with more advanced tuning and training (e.g. training on more epochs) this configuration allowed us to achieve near-perfect performance on the training and holdout sets and be able to train within reasonable time horizons.  

100 hidden units is the starting point. The other brute force runs (same words each iteration), we will decrease the number of hidden units by 20 succesively: 100, 80, 60, 40, 20.

This procedure here differs from `brute_force_1` because we write out the test data for every word at the end of training. Not just the aggregate test data for the train and test sets respectively.

In [3]:
with open('data/params.json', 'r') as f:
    cfg = json.load(f)

## Inputs and outputs

In [4]:
X = np.genfromtxt('data/kidwords/orth-kid.csv', delimiter=",")
Y = np.genfromtxt('data/kidwords/phon-kid.csv', delimiter=",")

words = pd.read_csv('data/kidwords/kidwords.csv', header=None)[0].tolist()

Obtain frequencies for the frequency-weighting operation

In [5]:
elp = pd.read_csv('~/research/words/elp/elp_full_5.27.16.csv')

In [6]:
frequencies = {}

for word in words:
    rowmatch = elp[elp['Word']==word]
    if not rowmatch.empty:
        frequencies[word] = rowmatch['Freq_HAL'].values[0]+1
    else:
        frequencies[word] = 1

In [7]:
frequencies_ = [frequencies[word] for word in words]

In [8]:
weights = np.array([scale(frequency, cfg["K"]) for frequency in frequencies_])



plt.hist(weights)
plt.show(color = "skyblue", edgecolor = "black")

## 100 hidden units

In [9]:
hidden = 100
PATH = 'outputs/brute_force_2_with_frequency/'

start = time.time()

for sample in range(samples.shape[1]):
        sfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_summary.csv'
        pfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_preds.csv'
        afn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_accuracies.csv'

        with open(PATH + sfn, 'w') as f:
            f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                    "hidden_units",
                                                    "learning_rate",
                                                    "batch_size",
                                                    "epochs",
                                                    "loss_train",
                                                    "accuracy_train",
                                                    "mse_train",
                                                    "loss_test",
                                                    "accuracy_test",
                                                    "mse_test",
                                                    "loss_holdout",
                                                    "accuracy_holdout",
                                                    "mse_holdout"))

            model = learner(X, Y, cfg['seed'], hidden, optimizer=Adam(learning_rate=cfg['learning_rate']))
            model.fit(X[samples[:, sample]], Y[samples[:, sample]], epochs=cfg['epochs'], batch_size=cfg['batch_size'], verbose=False, sample_weight = weights[samples[:, sample]])

            loss_train, accuracy_train, mse_train = model.evaluate(X[samples[:, sample]], Y[samples[:, sample]], verbose=0) 
            loss_test, accuracy_test, mse_test = model.evaluate(X[tests[:, sample]], Y[tests[:, sample]], verbose=0) 
            loss_holdout, accuracy_holdout, mse_holdout = model.evaluate(X[holdouts[:, sample]], Y[holdouts[:, sample]], verbose=0) 

            preds = (model.predict(X) > .5).astype(int)
            np.savetxt(PATH + pfn, preds, fmt='%d', delimiter=',')

            accuracies = (preds == Y).astype(int)
            np.savetxt(PATH + afn, np.mean(accuracies, axis = 1), delimiter=',', fmt='%0.5f')

            f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                        hidden,
                                                        cfg['learning_rate'],
                                                        cfg['batch_size'],
                                                        cfg['epochs'],
                                                        loss_train,
                                                        accuracy_train,
                                                        mse_train,
                                                        loss_test,
                                                        accuracy_test,
                                                        mse_test,
                                                        loss_holdout,
                                                        accuracy_holdout,
                                                        mse_holdout))
end = time.time()
print(round(end-start, 4), "seconds elapsed")

4038.1323 seconds elapsed


## 80 Hidden Units

In [ ]:
hidden = 80
PATH = 'outputs/brute_force_2_with_frequency/'

start = time.time()

for sample in range(samples.shape[1]):

    sfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_summary.csv'
    pfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_preds.csv'
    afn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_accuracies.csv'
    
    with open(PATH + sfn, 'w') as f:
        f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                "hidden_units",
                                                "learning_rate",
                                                "batch_size",
                                                "epochs",
                                                "loss_train",
                                                "accuracy_train",
                                                "mse_train",
                                                "loss_test",
                                                "accuracy_test",
                                                "mse_test",
                                                "loss_holdout",
                                                "accuracy_holdout",
                                                "mse_holdout"))

        model = learner(X, Y, cfg['seed'], hidden, optimizer=Adam(learning_rate=cfg['learning_rate']))
        model.fit(X[samples[:, sample]], Y[samples[:, sample]], epochs=cfg['epochs'], batch_size=cfg['batch_size'], verbose=False, sample_weight = weights[samples[:, sample]])

        loss_train, accuracy_train, mse_train = model.evaluate(X[samples[:, sample]], Y[samples[:, sample]], verbose=0) 
        loss_test, accuracy_test, mse_test = model.evaluate(X[tests[:, sample]], Y[tests[:, sample]], verbose=0) 
        loss_holdout, accuracy_holdout, mse_holdout = model.evaluate(X[holdouts[:, sample]], Y[holdouts[:, sample]], verbose=0) 

        preds = (model.predict(X) > .5).astype(int)
        np.savetxt(PATH + pfn, preds, fmt='%d', delimiter=',')

        accuracies = (preds == Y).astype(int)
        np.savetxt(PATH + afn, np.mean(accuracies, axis = 1), delimiter=',', fmt='%0.5f')

        f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                    hidden,
                                                    cfg['learning_rate'],
                                                    cfg['batch_size'],
                                                    cfg['epochs'],
                                                    loss_train,
                                                    accuracy_train,
                                                    mse_train,
                                                    loss_test,
                                                    accuracy_test,
                                                    mse_test,
                                                    loss_holdout,
                                                    accuracy_holdout,
                                                    mse_holdout))
end = time.time()
print(round(end-start, 4), "seconds elapsed")

## 60 Hidden Units

In [ ]:
hidden = 60
PATH = 'outputs/brute_force_2_with_frequency/'

start = time.time()

for sample in range(samples.shape[1]):

    sfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_summary.csv'
    pfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_preds.csv'
    afn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_accuracies.csv'
    
    with open(PATH + sfn, 'w') as f:
        f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                "hidden_units",
                                                "learning_rate",
                                                "batch_size",
                                                "epochs",
                                                "loss_train",
                                                "accuracy_train",
                                                "mse_train",
                                                "loss_test",
                                                "accuracy_test",
                                                "mse_test",
                                                "loss_holdout",
                                                "accuracy_holdout",
                                                "mse_holdout"))

        model = learner(X, Y, cfg['seed'], hidden, optimizer=Adam(learning_rate=cfg['learning_rate']))
        model.fit(X[samples[:, sample]], Y[samples[:, sample]], epochs=cfg['epochs'], batch_size=cfg['batch_size'], verbose=False, sample_weight = weights[samples[:, sample]])

        loss_train, accuracy_train, mse_train = model.evaluate(X[samples[:, sample]], Y[samples[:, sample]], verbose=0) 
        loss_test, accuracy_test, mse_test = model.evaluate(X[tests[:, sample]], Y[tests[:, sample]], verbose=0) 
        loss_holdout, accuracy_holdout, mse_holdout = model.evaluate(X[holdouts[:, sample]], Y[holdouts[:, sample]], verbose=0) 

        preds = (model.predict(X) > .5).astype(int)
        np.savetxt(PATH + pfn, preds, fmt='%d', delimiter=',')

        accuracies = (preds == Y).astype(int)
        np.savetxt(PATH + afn, np.mean(accuracies, axis = 1), delimiter=',', fmt='%0.5f')
        
        f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                    hidden,
                                                    cfg['learning_rate'],
                                                    cfg['batch_size'],
                                                    cfg['epochs'],
                                                    loss_train,
                                                    accuracy_train,
                                                    mse_train,
                                                    loss_test,
                                                    accuracy_test,
                                                    mse_test,
                                                    loss_holdout,
                                                    accuracy_holdout,
                                                    mse_holdout))
end = time.time()
print(round(end-start, 4), "seconds elapsed")

## 40 Hidden Units

In [ ]:
hidden = 40
PATH = 'outputs/brute_force_2_with_frequency/'

start = time.time()

for sample in range(samples.shape[1]):

    sfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_summary.csv'
    pfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_preds.csv'
    afn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_accuracies.csv'
    
    with open(PATH + sfn, 'w') as f:
        f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                "hidden_units",
                                                "learning_rate",
                                                "batch_size",
                                                "epochs",
                                                "loss_train",
                                                "accuracy_train",
                                                "mse_train",
                                                "loss_test",
                                                "accuracy_test",
                                                "mse_test",
                                                "loss_holdout",
                                                "accuracy_holdout",
                                                "mse_holdout"))

        model = learner(X, Y, cfg['seed'], hidden, optimizer=Adam(learning_rate=cfg['learning_rate']))
        model.fit(X[samples[:, sample]], Y[samples[:, sample]], epochs=cfg['epochs'], batch_size=cfg['batch_size'], verbose=False, sample_weight = weights[samples[:, sample]])

        loss_train, accuracy_train, mse_train = model.evaluate(X[samples[:, sample]], Y[samples[:, sample]], verbose=0) 
        loss_test, accuracy_test, mse_test = model.evaluate(X[tests[:, sample]], Y[tests[:, sample]], verbose=0) 
        loss_holdout, accuracy_holdout, mse_holdout = model.evaluate(X[holdouts[:, sample]], Y[holdouts[:, sample]], verbose=0) 

        preds = (model.predict(X) > .5).astype(int)
        np.savetxt(PATH + pfn, preds, fmt='%d', delimiter=',')

        accuracies = (preds == Y).astype(int)
        np.savetxt(PATH + afn, np.mean(accuracies, axis = 1), delimiter=',', fmt='%0.5f')

        f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                    hidden,
                                                    cfg['learning_rate'],
                                                    cfg['batch_size'],
                                                    cfg['epochs'],
                                                    loss_train,
                                                    accuracy_train,
                                                    mse_train,
                                                    loss_test,
                                                    accuracy_test,
                                                    mse_test,
                                                    loss_holdout,
                                                    accuracy_holdout,
                                                    mse_holdout))
end = time.time()
print(round(end-start, 4), "seconds elapsed")

## 20 Hidden Units

In [10]:
hidden = 20
PATH = 'outputs/brute_force_2_with_frequency/'

start = time.time()

for sample in range(samples.shape[1]):
    if sample > 5810:
        sfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_summary.csv'
        pfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_preds.csv'
        afn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_accuracies.csv'

        with open(PATH + sfn, 'w') as f:
            f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                    "hidden_units",
                                                    "learning_rate",
                                                    "batch_size",
                                                    "epochs",
                                                    "loss_train",
                                                    "accuracy_train",
                                                    "mse_train",
                                                    "loss_test",
                                                    "accuracy_test",
                                                    "mse_test",
                                                    "loss_holdout",
                                                    "accuracy_holdout",
                                                    "mse_holdout"))

            model = learner(X, Y, cfg['seed'], hidden, optimizer=Adam(learning_rate=cfg['learning_rate']))
            model.fit(X[samples[:, sample]], Y[samples[:, sample]], epochs=cfg['epochs'], batch_size=cfg['batch_size'], verbose=False, sample_weight = weights[samples[:, sample]])

            loss_train, accuracy_train, mse_train = model.evaluate(X[samples[:, sample]], Y[samples[:, sample]], verbose=0) 
            loss_test, accuracy_test, mse_test = model.evaluate(X[tests[:, sample]], Y[tests[:, sample]], verbose=0) 
            loss_holdout, accuracy_holdout, mse_holdout = model.evaluate(X[holdouts[:, sample]], Y[holdouts[:, sample]], verbose=0) 

            preds = (model.predict(X) > .5).astype(int)
            np.savetxt(PATH + pfn, preds, fmt='%d', delimiter=',')

            accuracies = (preds == Y).astype(int)
            np.savetxt(PATH + afn, np.mean(accuracies, axis = 1), delimiter=',', fmt='%0.5f')

            f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                        hidden,
                                                        cfg['learning_rate'],
                                                        cfg['batch_size'],
                                                        cfg['epochs'],
                                                        loss_train,
                                                        accuracy_train,
                                                        mse_train,
                                                        loss_test,
                                                        accuracy_test,
                                                        mse_test,
                                                        loss_holdout,
                                                        accuracy_holdout,
                                                        mse_holdout))
        

    

end = time.time()
print(round(end-start, 4), "seconds elapsed")

12922.6 seconds elapsed


# 5 hidden units

In [ ]:
hidden = 5
PATH = 'outputs/brute_force_2_with_frequency/'

start = time.time()

for sample in range(samples.shape[1]):
    if sample > 5810:
        sfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_summary.csv'
        pfn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_preds.csv'
        afn = 'sample_' + str(sample) + '_hidden_' + str(hidden) + '_accuracies.csv'

        with open(PATH + sfn, 'w') as f:
            f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                    "hidden_units",
                                                    "learning_rate",
                                                    "batch_size",
                                                    "epochs",
                                                    "loss_train",
                                                    "accuracy_train",
                                                    "mse_train",
                                                    "loss_test",
                                                    "accuracy_test",
                                                    "mse_test",
                                                    "loss_holdout",
                                                    "accuracy_holdout",
                                                    "mse_holdout"))

            model = learner(X, Y, cfg['seed'], hidden, optimizer=Adam(learning_rate=cfg['learning_rate']))
            model.fit(X[samples[:, sample]], Y[samples[:, sample]], epochs=cfg['epochs'], batch_size=cfg['batch_size'], verbose=False, sample_weight = weights[samples[:, sample]])

            loss_train, accuracy_train, mse_train = model.evaluate(X[samples[:, sample]], Y[samples[:, sample]], verbose=0) 
            loss_test, accuracy_test, mse_test = model.evaluate(X[tests[:, sample]], Y[tests[:, sample]], verbose=0) 
            loss_holdout, accuracy_holdout, mse_holdout = model.evaluate(X[holdouts[:, sample]], Y[holdouts[:, sample]], verbose=0) 

            preds = (model.predict(X) > .5).astype(int)
            np.savetxt(PATH + pfn, preds, fmt='%d', delimiter=',')

            accuracies = (preds == Y).astype(int)
            np.savetxt(PATH + afn, np.mean(accuracies, axis = 1), delimiter=',', fmt='%0.5f')

            f.write("{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                        hidden,
                                                        cfg['learning_rate'],
                                                        cfg['batch_size'],
                                                        cfg['epochs'],
                                                        loss_train,
                                                        accuracy_train,
                                                        mse_train,
                                                        loss_test,
                                                        accuracy_test,
                                                        mse_test,
                                                        loss_holdout,
                                                        accuracy_holdout,
                                                        mse_holdout))
        

    
end = time.time()
print(round(end-start, 4), "seconds elapsed")